#Atividade 2 - Programação Matemática
##Alunos
 - Daniel Messias Santos - 202110168
 - Thiago Pereira Freire - 202110167

# Enunciado
Uma certa fábrica de camisetas deseja aproveitar as finais de um campeonato de futebol para vender camisetas dos times envolvidos. Os jogos vão durar quatro semanas. O custo de produção de cada camiseta é de R\$ 2,00 nas duas primeiras semanas e R\$ 2,50 nas duas últimas, quando a concorrência demandar por material no mercado. A demanda semanal de camisetas será de 5.000, 10.000, 30.000 e 60.000. A capacidade máxima de produção da empresa é de 25.000 camisetas semanalmente. Na primeira e na segunda semanas, a empresa poderá contratar horas extras de serviço e fabricar mais 10.000 camisetas em cada semana. Nesse caso, o custo de produção sobe para R\$ 2,80. O excesso de produção pode ser estocado a um custo de R\$ 0,20 por unidade por semana.

Formule um modelo que minimize os custos.

# Modelo de Minimização de Custos para Produção de Camisetas

## Variáveis de Decisão
- \( $P_t$ \): quantidade de camisetas produzidas na semana \( $t$ \) (onde \( $t$ = 1, 2, 3, 4 \))
- \( $E_t$ \): quantidade de camisetas produzidas com horas extras na semana \( $t$ \) (onde \( $t$ = 1, 2 \))
- \( $I_t$ \): quantidade de camisetas em estoque no final da semana \( t \) (onde \( $t$ = 1, 2, 3, 4 \))

## Parâmetros
- Demanda semanal \( $D_t$ \):
  - \( $D_1$ = 5000 \)
  - \( $D_2$ = 10000 \)
  - \( $D_3$ = 30000 \)
  - \( $D_4$ = 60000 \)

- Custos de produção:
  - Semanas 1 e 2: R\$ 2,00
  - Semanas 3 e 4: R\$ 2,50
  - Custo com horas extras: R$ 2,80 (apenas nas semanas 1 e 2)

- Custo de estocagem: R$ 0,20 por unidade por semana.

- Capacidade de produção:
  - Sem horas extras: 25.000 camisetas por semana.
  - Com horas extras: 35.000 camisetas por semana (25.000 + 10.000).

## Função Objetivo
Minimizar o custo total:
\begin{equation}
Z = \sum_{t=1}^{4} C_t \cdot P_t + \sum_{t=1}^{2} (2.80 \cdot E_t) + \sum_{t=1}^{4} (0.20 \cdot I_t)
\end{equation}

onde:
- \( $C_t$ \) é o custo de produção na semana \( $t$ \):
  - \( $C_1$ = 2.00 \)
  - \( $C_2$ = 2.00 \)
  - \( $C_3$ = 2.50 \)
  - \( $C_4$ = 2.50 \)

## Restrições

1. **Capacidade de produção**:
   $P_t \leq 25000, \quad\text{para } t = 1, 2, 3, 4$

2. **Capacidade de produção com horas extras**:
   $P_t \leq 10000, \quad \text{para } t = 1, 2$

3. **Demandas**:
   $P_t + I_{t-1} - I_t = D_t, \quad \text{para } t = 1, 2, 3, 4$
   (com $I_0$ = 0 )

4. **Estoque não negativo**:
   $I_t \geq 0, \quad \text{para } t = 1, 2, 3, 4$

5. **Produção não negativa**:
   $P_t \geq 0, \quad \text{para } t = 1, 2, 3, 4$
   $E_t \geq 0, \quad \text{para } t = 1, 2$

### Importação da biblioteca

In [1]:
!pip install pulp
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 28.0 MB/s eta 0:00:00


### Parâmetros

In [2]:
# Definição de parametros:
periodos = [1, 2, 3, 4]
demanda = {1: 5000, 2: 10000, 3: 30000, 4: 60000}
custosProducao = {1: 2.00, 2: 2.00, 3: 2.50, 4: 2.50}
capacidadeProducao = 25000
capacidadeProducaoHorasExtras = 10000

### Criação do modelo

In [3]:
modelo = pulp.LpProblem("ProducaoCamisetas", pulp.LpMinimize)

### Variáveis de decisão

In [4]:
qtdProduzida = pulp.LpVariable.dicts("qtdProduzida", periodos, lowBound=0, cat='Integer')
qtdHorasExtras = pulp.LpVariable.dicts("qtdHorasExtras", [1, 2], lowBound=0, cat='Integer')
qtdEmEstoque = pulp.LpVariable.dicts("qtdEmEstoque", periodos, lowBound=0, cat='Integer')

### Função objetivo

In [5]:
modelo += pulp.lpSum(custosProducao[t] * qtdProduzida[t] for t in periodos) + pulp.lpSum(2.80 * qtdHorasExtras[t] for t in [1, 2]) + pulp.lpSum(0.2 * qtdEmEstoque[t] for t in periodos)

## Restrições

In [6]:
# Restrições
for t in periodos:
    modelo += qtdProduzida[t] <= capacidadeProducao
    modelo += qtdEmEstoque[t] >= 0
    modelo += qtdProduzida[t] >= 0

    if t in [1, 2]:
        modelo += qtdHorasExtras[t] <= capacidadeProducaoHorasExtras
        modelo += qtdHorasExtras[t] >= 0

    if t == 1:
        modelo += qtdProduzida[t] + qtdHorasExtras[t] - qtdEmEstoque[t] == demanda[t]
    elif t == 2:
        modelo += qtdProduzida[t] + qtdHorasExtras[t] + qtdEmEstoque[t-1] - qtdEmEstoque[t] == demanda[t]
    else:
        modelo += qtdProduzida[t] + qtdEmEstoque[t-1] - qtdEmEstoque[t] == demanda[t]

### Solver

In [7]:
status = modelo.solve()

### Impressão dos resultados

In [9]:
print(f"Status: {pulp.LpStatus[status]}")
for i in periodos:
    print(f"Semana {i}: ",end='')
    print(f" Camisetas produzidas: {qtdProduzida[i].value()}", end='\t')
    if i in [1, 2]:
        print(f"Quantidade de camisetas extras: {qtdHorasExtras[i].value()}", end='\t')
    print(f"Estoque:  {qtdEmEstoque[i].value()}")

print(f"Custo: {modelo.objective.value()}")

Status: Optimal
Semana 1:  Camisetas produzidas: 25000.0	Quantidade de camisetas extras: 0.0	Estoque:  20000.0
Semana 2:  Camisetas produzidas: 25000.0	Quantidade de camisetas extras: 5000.0	Estoque:  40000.0
Semana 3:  Camisetas produzidas: 25000.0	Estoque:  35000.0
Semana 4:  Camisetas produzidas: 25000.0	Estoque:  0.0
Custo: 258000.0
